<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2024 Daniela Concha. Todos los derechos reservados.</font>
</p>

# Tabla de contenidos

1. [Ejemplos de Iterable e Iterador](#Ejemplos_de_Iterable_e_Iterador)
    1. [Los efectos del **Iterable** se ven afectador por el **Iterador**](#Los_efectos_del_Iterable_se_ven_afectador_por_el_Iterador)
    2. [Opción 1: Hacer uso de `deepcopy`](#Opción_1_Hacer_uso_de_deepcopy)
    3. [Opción 2: Solo copiar los elementos necesarios](#Opción_2_Solo_copiar_los_elementos_necesarios)
2. [Beneficios de utilizar un Iterables e Iteradores personalizados](#Beneficios_de_utilizar_un_Iterables_e_Iteradores_personalizados)
    1. [Recorrer de izquierda a derecha](#Recorrer_de_izquierda_a_derecha)
    1. [Recorrer de derecha a izquierda](#Recorrer_de_derecha_a_izquierda)
    1. [Recorrer de forma alfabética](#Recorrer_de_forma_alfabética)
    1. [Recorrer de forma aleatoria](#Recorrer_de_forma_aleatoria)

## Ejemplos de Iterable e Iterador

Los siguientes códigos buscan poner en ejecución el código presentado en la clase de esta semana (Semana 05 - Modelación OOP e Iterables (Cartas)), resaltando la importancia de hacer que el **Iterador** no afecte a los elementos del **Iterable**.

### Los efectos del **Iterable** se ven afectador por el **Iterador**

In [1]:
from copy import copy
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self)

class RecorredorDeCartas:
    def __init__(self, recorrible: RecorribleDeCartas) -> None:
        # Para no modificar original, se hace una copy del recorrible.
        self.recorrible = copy(recorrible)

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.recorrible.cartas:
            raise StopIteration("Sin cartas")

        cartas = self.recorrible.cartas
        proxima_carta = cartas.pop(0)
        return proxima_carta

In [2]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])
iterador_mazo = iter(mazo)

print(next(iterador_mazo))
print(next(iterador_mazo))
print(next(iterador_mazo))

AS
J
Q


Si iteramos sobre el `RecorribleDeCartas`, podremos notar que obtendremos un iterador (`RecorredorDeCartas`) y que podremos obtener cada carta del maso. 

Pero si revisamos las cartas del mazo original (`mazo`), podremos notar que las cartas fueros modificadas 😱:

In [3]:
print('Cartas del iterable del mazo:', mazo.cartas)
print('Cartas del iterador del mazo:', iterador_mazo.recorrible.cartas)

Cartas del iterable del mazo: ['K']
Cartas del iterador del mazo: ['K']


¿Qué causa lo anterior? 
> Cuando se hace la copia del `RecorribleDeCartas`
> ```python
> self.recorrible = copy(recorrible)
> ```
> estamos haciendo una "_soft copy_" de la instancia, por lo que solo se crea una nueva instancia con los mismos atributos de la original, pero no se hace una copia profunda de los mismos, por lo que la lista de cartas (`cartas`) de `RecorribleDeCartas` será la misma que la del `RecordorDeCartas`.

Entonces, ¿cómo podemos evitar lo anterior?

### Opción 1: Hacer uso de `deepcopy`

Al hacer uso de `deepcopy` para generar la copia de la instancia de `RecorribleDeCartas`, entonces nos aseguraremos que sus cartas no sean la misma instancia que las del `RecordorDeCartas`.

In [4]:
from copy import deepcopy
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self)

class RecorredorDeCartas:
    def __init__(self, recorrible: RecorribleDeCartas) -> None:
        # Para no modificar original, se hace una deepcopy del recorrible.
        self.recorrible = deepcopy(recorrible)

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.recorrible.cartas:
            raise StopIteration("Sin cartas")

        cartas = self.recorrible.cartas
        proxima_carta = cartas.pop(0)
        return proxima_carta

In [5]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])
iterador_mazo = iter(mazo)

print(next(iterador_mazo))
print(next(iterador_mazo))
print(next(iterador_mazo))

print()

print('Cartas del iterable del mazo:', mazo.cartas)
print('Cartas del iterador del mazo:', iterador_mazo.recorrible.cartas)

AS
J
Q

Cartas del iterable del mazo: ['AS', 'J', 'Q', 'K']
Cartas del iterador del mazo: ['K']


### Opción 2: Solo copiar los elementos necesarios

Otra alternativa, es solo copiar los elementos que son necesarios para el recorrible, en este caso, el listado de cartas.

In [6]:
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            # Para evitar entregar la instancia de RecorribleDeCartas,
            # solo se entrega el listado de cartas.
            return RecorredorDeCartas(self.cartas)

class RecorredorDeCartas:
    def __init__(self, recorrible: List[Carta]) -> None:
        # Para no modificar original, se hace una copia del listado de cartas.
        self.cartas = recorrible.copy()

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.cartas:
            raise StopIteration("Sin cartas")

        proxima_carta = self.cartas.pop(0)
        return proxima_carta

In [7]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])
iterador_mazo = iter(mazo)

print(next(iterador_mazo))
print(next(iterador_mazo))
print(next(iterador_mazo))

print()

print('Cartas del iterable del mazo:', mazo.cartas)
print('Cartas del iterador del mazo:', iterador_mazo.cartas)

AS
J
Q

Cartas del iterable del mazo: ['AS', 'J', 'Q', 'K']
Cartas del iterador del mazo: ['K']


## Beneficios de utilizar un Iterables e Iteradores personalizados

Uno de los beneficios de hacer itables personalizados es que nos permite modificar el comportamiento que utilizamos para recorrer los elementos del iterable.

Aprovechandonos de los ejemplos vistosn en clase (Semana 05 - Modelación OOP e Iterables (Cartas)), podemos recorrer el mazo de cartas de distintas formas:

### Recorrer de izquierda a derecha

In [8]:
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self.cartas)

class RecorredorDeCartas:
    def __init__(self, recorrible: List[Carta]) -> None:
        self.cartas = recorrible.copy()

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.cartas:
            raise StopIteration("Sin cartas")

        # Obtenemos la primera carta de la lista y la sacamos.
        proxima_carta = self.cartas.pop(0)
        return proxima_carta

In [9]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])

for carta in mazo:
    print(carta)

AS
J
Q
K


### Recorrer de derecha a izquierda

In [10]:
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self.cartas)

class RecorredorDeCartas:
    def __init__(self, recorrible: List[Carta]) -> None:
        self.cartas = recorrible.copy()

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.cartas:
            raise StopIteration("Sin cartas")

        # Obtenemos la última carta de la lista y la sacamos.
        proxima_carta = self.cartas.pop(-1)
        return proxima_carta

In [11]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])

for carta in mazo:
    print(carta)

K
Q
J
AS


### Recorrer las cartas por orden alfabético

In [12]:
from typing import Any, List, Self


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas
        # Ordenamos las cartas.
        self.cartas.sort()

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self.cartas)

class RecorredorDeCartas:
    def __init__(self, recorrible: List[Carta]) -> None:
        self.cartas = recorrible.copy()

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.cartas:
            raise StopIteration("Sin cartas")

        # Obtenemos la primera carta de la lista y la sacamos.
        proxima_carta = self.cartas.pop(0)
        return proxima_carta

In [13]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])

for carta in mazo:
    print(carta)

AS
J
K
Q


### Recorrer de forma aleatoria

In [14]:
from typing import Any, List, Self
from random import randint


class Carta(str):
    pass

class RecorribleDeCartas:
    def __init__(self, cartas: List[Carta]) -> None:
        self.cartas = cartas

    def __iter__(self) -> Any:
            return RecorredorDeCartas(self.cartas)

class RecorredorDeCartas:
    def __init__(self, recorrible: List[Carta]) -> None:
        self.cartas = recorrible.copy()

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> Any:
        if not self.cartas:
            raise StopIteration("Sin cartas")

        # Obtenemos un número al azar entre 0 y el largo de la lista,
        # y sacamos dicho elemento de la lista.
        n = randint(0, len(self.cartas) -1)
        proxima_carta = self.cartas.pop(n)
        return proxima_carta

In [15]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])

for carta in mazo:
    print(carta)

Q
J
AS
K


Finalmente, no recuerden que cada `Iterador` o `Recorredor` son instancias independientes, por lo que podemos utilizar cada unos de estos para recorrer de forma simultánea el `Iterable` o `Recorrible` sin que estos se afecten entre sí:

In [16]:
mazo = RecorribleDeCartas([Carta('AS'), Carta('J'), Carta('Q'), Carta('K')])

iterador_1 = iter(mazo)
iterador_2 = iter(mazo)

print('Iteramos sobre el iterador 1:', next(iterador_1))
print('Iteramos sobre el iterador 2:', next(iterador_2))
print('Iteramos sobre el iterador 2:', next(iterador_2))
print('Iteramos sobre el iterador 1:', next(iterador_1))
print('Iteramos sobre el iterador 2:', next(iterador_2))

print()

print('Listado de cartas del iterador 1:', iterador_1.cartas)
print('Listado de cartas del iterador 2:', iterador_2.cartas)

Iteramos sobre el iterador 1: Q
Iteramos sobre el iterador 2: AS
Iteramos sobre el iterador 2: K
Iteramos sobre el iterador 1: AS
Iteramos sobre el iterador 2: J

Listado de cartas del iterador 1: ['J', 'K']
Listado de cartas del iterador 2: ['Q']
